In [6]:
import pandas as pd
import sys
import os

In [ ]:
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_path not in sys.path:
    sys.path.append(project_path)

import utils.get_data_utils as gdu

In [8]:
# Get the list of years for analysis
time_period = [ str(t) for t in range(2005, 2024)]

# Get the list of countries for analysis
country_codes = [
    "BE",  # Belgium
    "EL",  # Greece
    "LT",  # Lithuania
    "PT",  # Portugal
    "BG",  # Bulgaria
    "ES",  # Spain
    "LU",  # Luxembourg
    "RO",  # Romania
    "CZ",  # Czechia
    "FR",  # France
    "HU",  # Hungary
    "SI",  # Slovenia
    "DK",  # Denmark
    "HR",  # Croatia
    "MT",  # Malta
    "SK",  # Slovakia
    "DE",  # Germany
    "IT",  # Italy
    "NL",  # Netherlands
    "FI",  # Finland
    "EE",  # Estonia
    "CY",  # Cyprus
    "AT",  # Austria
    "SE",  # Sweden
    "IE",  # Ireland
    "LV",  # Latvia
    "PL"   # Poland
]


# Get the list of indicators along with their filters for our analysis
indicators = {
    # 'prc_hpi_a': {
    #     "unit": "RCH_A_AVG", # Annual average rate of change
    #     "purchase": "TOTAL",
    #     "label": "house_price_index"
    # },  # House price index
    # 'tipsho60': {
    #     "unit": "PTIR_LT_AVG", # Price-to-income ratio relative to long-term average
    #     "label": "price_income_ratio"
    # }, # Standardised house price-to-income ratio
    # 'sts_cobp_a': {
    #     "unit": "I21", # Index, 2021=100
    #     "indic_bt": "BPRM_SQM", # Building permits - m2 of useful floor area
    #     "cpa2_1": "CPA_F41001", # Residential buildings
    #     "label": "building_permits"
    # }, # Building permits
    # 'migr_imm1ctz': {
    #     "citizen": "TOTAL",
    #     "agedef": "REACH",
    #     "age": "TOTAL",
    #     "sex": "T",
    #     "label": "immigration"
    # }, # Immigration by age group, sex and citizenship
    # 'ilc_lvps08': {
    #     "sex": "T",
    #     "age": "Y18-34", # From 18 to 34 years
    #     "label": "living_with_parents"
    # }, # Persons living with their parents or contributing/benefiting from the household income (population aged 18 to 34 years)
    # 'sts_copi_a': {
    #     "unit": "PCH_SM", # Percentage change compared to same period in previous year
    #     "indic_bt": "PRC_PRR", # Producer prices
    #     "label": "construction_prices"
    # }, # Construction producer prices or costs, new residential buildings
    # # 'crim_off_cat': {
    # #     "unit": "P_HTHAB" # Per hundred thousand inhabitants
    # # }, # Police-recorded offences by offence category
    # 'earn_nt_net': {
    #     "ecase": "P1_NCH_AW100", # Single person without children earning 100% of the average earning
    #     "currency": "EUR",
    #     "estruct": "NET", # Net earning
    #     "label": "net_earnings"
    # }, # Annual net earnings
    # 'ilc_li48': {
    #     "deg_urb": "TOTAL",
    #     "label": "poverty_rate"
    # }, # At-risk-of-poverty rate after deducting housing costs by degree of urbanisation
    # 'tps00003': {
    #     "label": "pop_density"
    # }, # Population density
    # 'tps00206': {
    #     "label": "marriage_rate"
    # }, # Crude marriage rate
    'prc_hicp_midx': {
        "unit": "I15",
        "coicop": "CP00",
        "freq": "M",
        "label": "inflation_rate"
    }, # HICP - inflation rate
    'tps00019': {
        "indic_de": "GROWRT", # Crude rate of total population change
        "label": "pop_change"
    }, # Population change - crude rates of total change, natural change and net migration plus adjustment
    'tesem120': {
        "sex": "T",
        "label": "unemployment_rate"
    }, # Unemployment rate by sex
    # 'ert_bil_eur_a': {
    #     "unit": ""
    # }, # Euro/ECU exchange rates - annual data
}


for indicator in indicators:
    indicators[indicator]['geo'] = country_codes
    indicators[indicator]['time'] = time_period
    if 'freq' not in indicators[indicator]:
        indicators[indicator]['freq'] = 'A'

print(indicators)

{'prc_hicp_midx': {'unit': 'I15', 'coicop': 'CP00', 'freq': 'M', 'label': 'inflation_rate', 'geo': ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT', 'NL', 'FI', 'EE', 'CY', 'AT', 'SE', 'IE', 'LV', 'PL'], 'time': ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']}, 'tps00019': {'indic_de': 'GROWRT', 'label': 'pop_change', 'geo': ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT', 'NL', 'FI', 'EE', 'CY', 'AT', 'SE', 'IE', 'LV', 'PL'], 'time': ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'], 'freq': 'A'}, 'tesem120': {'sex': 'T', 'label': 'unemployment_rate', 'geo': ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT'

In [ ]:
df = gdu.get_eurostat_data(indicators)
print("Data retrieved from Eurostat")

Data retrieved from Eurostat


In [28]:
df.to_csv("../data/raw/eu_analysis_data.csv", index=False)
print("Data saved to data/eu_analysis_data.csv")

Data saved to data/eu_analysis_data.csv


In [23]:
df.head()

indicator,geo,time,inflation_rate,pop_change,unemployment_rate
0,AT,2005,82.299167,NaN,NaN
1,AT,2006,83.687500,NaN,NaN
2,AT,2007,85.529167,NaN,NaN
3,AT,2008,88.287500,NaN,NaN
4,AT,2009,88.642500,NaN,NaN


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   geo                513 non-null    object 
 1   time               513 non-null    object 
 2   inflation_rate     513 non-null    float64
 3   pop_change         324 non-null    float64
 4   unemployment_rate  413 non-null    float64
dtypes: float64(3), object(2)
memory usage: 20.2+ KB


In [27]:
df.describe()

indicator,inflation_rate,pop_change,unemployment_rate
count,513.000000,324.000000,413.000000
mean,99.707502,2.840432,8.560048
std,12.719322,9.416605,4.457581
min,64.904167,-23.600000,2.000000
25%,92.191667,-2.625000,5.700000
50%,99.941667,2.500000,7.400000
75%,104.032500,6.525000,10.000000
max,160.593333,43.400000,27.800000
